# NBA Inflation Adjusted Per 75 Possessions Stat Calculator

This Jupyter Notebook will allow you to calculate Inflation Adjusted Per 75 Possessions, or IA /75, stats for any NBA player since the 1973-74 season. These are very useful stats for comparing NBA players across eras because they account for the sometimes very significant differences in pace and leaguewide volumes of stats over the years. It will tabulate a player's IA Points/75, IA Rebounds/75, IA Assists/75, IA Steals/75, IA Blocks/75, IA Turnovers/75, and relative true shooting percentage (a measure of era adjusted scoring efficiency), for each season in a specified range of seasons, as well as each of these stats over the entire range. In addition, I introduce a slightly modified version of Inflation Adjusted Per 75 Possessions stats known as Inflation and Pace Adjusted Per Game, or IPAPG, stats, which can be calculated for the same range of seasons and statistical categories. IPAPG stats are more useful that IA /75 stats in some insances because IPAPG stats tend to have higher values for players that play more minutes as per game stats do. More details about IPAPG stats are available towards the bottom of the notebook.




To run this notebook, you will need:

-Player total stats table from basketball-reference.com (as csv file)

-Player per 100 possessions stats table from basketball-reference.com (as csv file)

To obtain these files, you can go to a player's basketball reference page, go to the table you need (Totals and Per 100 Possessions), click "Share & Export" above the table, and click "Get table as CSV (For Excel)". In the "Data" folder of this Git repository, I have included some example player data. The files of the format "firstname_lastname.csv" are the total stats for a given player and the ones of the format "firstname_lastname_per100.csv" are the per 100 possessions stats for a given player. I included the data for approximately 10-15 of the greatest players of all time (since the 1973-74 season) at each position. If the player you want to calculate stats for is/was an all time great, I would reccommend taking a look there to see if I included him so you don't have to grab the files.

If you run all the cells in this notebook, it will first prompt you to upload these two files for your player of choice (first total stats then per 100 possessions stats), then it will prompt you to choose a start and end year in which to calculate the stats and the year you wish to adjust to (the present year is most commonly used), and finally it will display a series of tables and text outputs which are, in order, the team's possessions per game in each season in the range, the inflation adjustment factor for each stat in each season in the range, the IA /75 stats for each season in the range, the total IA /75 stats and cumulative infaltion adjusted stats for all seasons in the range, the IPAPG stats for each season in the range, and the total IPAPG stats and cumulative infaltion adjusted stats for all seasons in the range. Happy calculating!

### Upload total stats csv file

Running the cell below will prompt you to upload the total stats csv file for your player of choice. To obtain this file, you can go to a player's basketball reference page, go to the "Totals" section, click "Share & Export" above the table, and click "Get table as CSV (For Excel)". There are a few example players in the "Data" folder of this git repository. If you wish to use one of those players, upload the file of the format "firstname_lastname.csv" here.

In [1395]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

def upload_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    print("Upload player total stats (via Basketball Reference)")
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        try:
            player_stats = pd.read_csv(file_path)
            print("File successfully loaded.")
            return player_stats
        except Exception as e:
            print(f"Error loading file: {e}")
            return None
    else:
        print("No file selected.")
        return None

# Call the function to upload and read the file
player_stats = upload_file()

# Display the first few rows of the dataframe (if desired)
# if player_stats is not None:
#     print(player_stats.head())


Upload player total stats (via Basketball Reference)


2024-06-19 03:46:23.709 python[1674:12841] +[CATransaction synchronize] called within transaction
2024-06-19 03:46:23.844 python[1674:12841] +[CATransaction synchronize] called within transaction
2024-06-19 03:46:24.233 python[1674:12841] +[CATransaction synchronize] called within transaction
2024-06-19 03:46:24.828 python[1674:12841] +[CATransaction synchronize] called within transaction


File successfully loaded.


### Upload per 100 possessions stats csv file

Running the cell below will prompt you to upload the per 100 possessions stats csv file for your player of choice. To obtain this file, you can go to a player's basketball reference page, go to the "Per 100 Possessions" section, click "Share & Export" above the table, and click "Get table as CSV (For Excel)". There are a few example players in the "Data" folder of this git repository. If you wish to use one of those players, upload the file of the format "firstname_lastname_per100.csv" here.

In [1396]:
def upload_file_per100():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    print("Upload player per 100 possession stats (via Basketball Reference)")
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        try:
            player_stats_per100 = pd.read_csv(file_path)
            print("File successfully loaded.")
            return player_stats_per100
        except Exception as e:
            print(f"Error loading file: {e}")
            return None
    else:
        print("No file selected.")
        return None

# Call the function to upload and read the file
player_stats_per100 = upload_file_per100()

# Display the first few rows of the dataframe (if desired)
# if player_stats_per100 is not None:
#     print(player_stats_per100.head())


Upload player per 100 possession stats (via Basketball Reference)


2024-06-19 03:46:43.154 python[1674:12841] +[CATransaction synchronize] called within transaction
2024-06-19 03:46:43.454 python[1674:12841] +[CATransaction synchronize] called within transaction


File successfully loaded.


### Preprocessing

The cell below will preprocess the data that you have uploaded. Basketball Reference has multiple rows in their tables corresponding to a single season if a player plays for multiple teams in that season. This cell will make it so only the row with their stats from the entire year is kept. No action is required on your part other than running the cell.

In [1397]:
def preprocess_data(df):
    # Identify duplicate seasons
    duplicate_seasons = df[df.duplicated(subset=['Season'], keep=False)]
    
    # Separate the rows with multiple teams and single team
    multi_team_seasons = duplicate_seasons[duplicate_seasons['Tm'] == 'TOT']
    single_team_seasons = df.drop_duplicates(subset=['Season'], keep=False)
    
    # Combine the preprocessed DataFrames
    df_preprocessed = pd.concat([single_team_seasons, multi_team_seasons]).drop_duplicates(subset=['Season'])
    
    # Sort the DataFrame by season
    df_preprocessed = df_preprocessed.sort_values(by=['Season'])
    
    return df_preprocessed

# Preprocess the player_stats and player_stats_per100 DataFrames
player_stats = preprocess_data(player_stats)
player_stats_per100 = preprocess_data(player_stats_per100)

# Display the preprocessed DataFrames (if desired)
# print("Preprocessed player_stats DataFrame:")
# print(player_stats)

# print("\nPreprocessed player_stats_per100 DataFrame:")
# print(player_stats_per100)

### League Averages file

The cell below reads in the league averages csv file. It contains the points, rebounds, assists, etc that teams recorded on average in each year since 1973-74 per 100 possessions. This file is already in the "Data" folder of this git repository. No action is required of you other than running the cell, unless you want to get an updated version of this file, in which case you can find it on basketball-reference.com and you must name it "league_averages.csv" and put it in the "Data" folder of this repository.

In [1398]:
# Read the league_averages CSV file directly
try:
    league_averages_per100 = pd.read_csv("Data/league_averages.csv")
    print("League Averages Per 100 Possessions DataFrame successfully loaded.")
    
    # Remove all years before 1973-74 (No per 100 possessions data available until 1973-74)
    league_averages_per100 = league_averages_per100[league_averages_per100['Season'] >= '1973-74']
except Exception as e:
    print(f"Error loading league_averages.csv file: {e}")
    league_averages_per100 = None

# Display the first few rows of the league_averages dataframe (if desired)
# if league_averages_per100 is not None:
#     print("League Averages Per 100 Possessions DataFrame:")
#     print(league_averages_per100.head())


League Averages Per 100 Possessions DataFrame successfully loaded.


### First Season in Range

Running the cell below will generate a widget prompting you to select the first season in the desired range in which to calculate stats. Once you have clicked "Select", the widget may not disappear. If it doesn't, just ignore it and run the next cell, then it will disappear. If this cell is running for more than 5 seconds, terminate it and run it again, it will almost certainly run instantly the next time.

In [1399]:
import tkinter as tk
from tkinter import ttk

def select_start_season(player_stats):
    def on_select():
        nonlocal selected_start_season
        selected_start_season = start_season_menu.get()
        if selected_start_season:
            root.quit()  # Terminate the GUI event loop
            root.destroy()  # Destroy the GUI window
        else:
            print("No season selected.")
    
    # Get unique seasons from the player_stats DataFrame, filtering out invalid entries and before 1973-74
    seasons = player_stats['Season'].dropna().astype(str)
    seasons = [season for season in seasons if '-' in season and season >= '1973-74']
    seasons = sorted(seasons)
    
    root = tk.Tk()
    root.title("Select Start Season")
    
    tk.Label(root, text="Start Season:").grid(row=0, column=0, padx=10, pady=10)
    
    start_season_var = tk.StringVar()
    start_season_menu = ttk.Combobox(root, textvariable=start_season_var, values=seasons)
    start_season_menu.grid(row=0, column=1, padx=10, pady=10)
    start_season_menu.bind('<<ComboboxSelected>>')
    
    tk.Button(root, text="Select", command=on_select).grid(row=1, columnspan=2, pady=10)
    
    selected_start_season = None
    root.mainloop()
    return selected_start_season

# Instantiate and print start season based on selection
start_season = select_start_season(player_stats)
print(f"Start Season: {start_season}")


Start Season: 2008-09


### Last Season in Range

Running the cell below will generate a widget prompting you to select the last season in the desired range in which to calculate stats. It will not work if you choose a season before the season you previously selected as the first season in the range. Once you have clicked "Select", the widget may not disappear. If it doesn't, just ignore it and run the next cell, then it will disappear. If this cell is running for more than 5 seconds, terminate it and run it again, it will almost certainly run instantly the next time.

In [1400]:
import tkinter as tk
from tkinter import ttk

def select_end_season(player_stats, start_season):
    def on_select():
        nonlocal selected_end_season
        selected_end_season = end_season_menu.get()
        if selected_end_season:
            if selected_end_season >= start_season:
                root.quit()  # Terminate the GUI event loop
                root.destroy()  # Destroy the GUI window
            else:
                print("End season must be on or after the start season.")
        else:
            print("No season selected.")
    
    # Get unique seasons from the player_stats DataFrame, filtering out invalid entries and before 1973-74
    seasons = player_stats['Season'].dropna().astype(str)
    seasons = [season for season in seasons if '-' in season and season >= '1973-74']
    seasons = sorted(seasons)
    
    root = tk.Tk()
    root.title("Select End Season")
    
    tk.Label(root, text="End Season:").grid(row=0, column=0, padx=10, pady=10)
    
    end_season_var = tk.StringVar()
    end_season_menu = ttk.Combobox(root, textvariable=end_season_var, values=seasons)
    end_season_menu.grid(row=0, column=1, padx=10, pady=10)
    end_season_menu.bind('<<ComboboxSelected>>')
    
    tk.Button(root, text="Select", command=on_select).grid(row=1, columnspan=2, pady=10)
    
    selected_end_season = None
    root.mainloop()
    return selected_end_season

# Instantiate and print end season based on selection
end_season = select_end_season(player_stats, start_season)
print(f"End Season: {end_season}")


End Season: 2013-14


### Select Season for Adjustment

Running the cell below will generate a widget prompting you to select the season to which you want to adjust the player's stats. It is most common to choose the current season for this for direct comparison with modern players, but you can choose any season since 1973-74. Once you have clicked "Select", the widget may not disappear. If it doesn't, just ignore it and run the next cell. It may still not disappear, but its presence will not affect any of the subsequent code. If this cell is running for more than 5 seconds, terminate it and run it again, it will almost certainly run instantly the next time.

In [1401]:
def select_inflation_adjustment_season(league_averages):
    def on_select():
        nonlocal selected_inflation_adjustment_season
        selected_inflation_adjustment_season = inflation_adjustment_menu.get()
        if selected_inflation_adjustment_season:
            root.quit()  # Terminate the GUI event loop
            root.destroy()  # Destroy the GUI window
        else:
            print("No season selected.")
    
    # Get unique seasons from the league_averages DataFrame, filtering out invalid entries
    seasons = league_averages['Season'].dropna().astype(str)
    seasons = [season for season in seasons if '-' in season]
    seasons = sorted(seasons)
    
    root = tk.Tk()
    root.title("Select Season for Inflation Adjustment")
    
    tk.Label(root, text="Season for Inflation Adjustment:").grid(row=0, column=0, padx=10, pady=10)
    
    inflation_adjustment_var = tk.StringVar()
    inflation_adjustment_menu = ttk.Combobox(root, textvariable=inflation_adjustment_var, values=seasons)
    inflation_adjustment_menu.grid(row=0, column=1, padx=10, pady=10)
    inflation_adjustment_menu.bind('<<ComboboxSelected>>')
    
    tk.Button(root, text="Select", command=on_select).grid(row=1, columnspan=2, pady=10)
    
    selected_inflation_adjustment_season = None
    root.mainloop()
    return selected_inflation_adjustment_season

# Instantiate and print inflation adjustment season based on selection
inflation_adjustment_season = select_inflation_adjustment_season(league_averages_per100)
print(f"Inflation Adjustment Season: {inflation_adjustment_season}")


Inflation Adjustment Season: 2023-24


### Team Possessions Per Game

Running the cell below will calculate and display the average number of possessions per game the player's team had in each season in the range. The formula for calculation is shown below.

possessions per game = total points * (48 / total minutes) * (100 / points per 100 possessions)

It is necessary for subsequent calculations. No action is required on your part other than running the cell.

In [1402]:
# Filter the data frames for the given range of seasons
filtered_player_stats = player_stats[(player_stats['Season'] >= start_season) & (player_stats['Season'] <= end_season)]
filtered_player_stats_per100 = player_stats_per100[(player_stats_per100['Season'] >= start_season) & (player_stats_per100['Season'] <= end_season)]

# Calculate the team possessions per game for each season
team_possessions_per_game = []

for season in filtered_player_stats['Season'].unique():
    total_points = filtered_player_stats[filtered_player_stats['Season'] == season]['PTS'].sum()
    total_minutes = filtered_player_stats[filtered_player_stats['Season'] == season]['MP'].sum()
    points_per_100 = filtered_player_stats_per100[filtered_player_stats_per100['Season'] == season]['PTS'].sum()
    
    if total_minutes > 0 and points_per_100 > 0:
        possessions_per_game = total_points * (48 / total_minutes) * (100 / points_per_100)
        team_possessions_per_game.append({
            'Season': season,
            'Team Possessions Per Game': possessions_per_game
        })

# Convert the result to a DataFrame
team_possessions_per_game_df = pd.DataFrame(team_possessions_per_game)

# Display the result
print("Team Possessions Per Game:")
print(team_possessions_per_game_df)


Team Possessions Per Game:
    Season  Team Possessions Per Game
0  2008-09                  88.755345
1  2009-10                  91.355361
2  2010-11                  90.882673
3  2011-12                  91.157135
4  2012-13                  90.583246
5  2013-14                  91.168133


### Stats to Adjust

The original stats for which this notebook calculates inflation and pace adjustments are points, rebounds, assists, steals, blocks, and turnovers. All of the subsequent code cells will have some form of these stats in the output.

### Inflation Adjustment Factors

Running the cell below will calculate the inflation adjustment factor for each stat for each season in the specified range. The formula for the inflation adjustment factors is shown below.

inflation adjustment factor = (inflation adjustment year league average stat) / (league average stat)

It will display these factors as a table so you can see how much more or less teams accumulate a given stat in each season in the specified range relative to the season chosen for inflation adjustment. No action is required on your part other than running the cell.

In [1403]:
from tabulate import tabulate

# Filter the data frames for the given range of seasons
filtered_player_stats = player_stats[(player_stats['Season'] >= start_season) & (player_stats['Season'] <= end_season)]
filtered_player_stats_per100 = player_stats_per100[(player_stats_per100['Season'] >= start_season) & (player_stats_per100['Season'] <= end_season)]

# Filter the league_averages_per100 for the specified inflation adjustment season
inflation_adjustment_league_avg = league_averages_per100[league_averages_per100['Season'] == inflation_adjustment_season]

# List of stats to calculate inflation adjustment for
stats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

# Dictionary to hold inflation adjustments for each stat
inflation_adjustments = {stat: [] for stat in stats}

# Loop through each season in the specified range
for season in filtered_player_stats['Season'].unique():
    league_avg_season = league_averages_per100[league_averages_per100['Season'] == season]
    
    if not league_avg_season.empty and not inflation_adjustment_league_avg.empty:
        for stat in stats:
            if stat in league_avg_season.columns and stat in inflation_adjustment_league_avg.columns:
                league_avg_stat = league_avg_season[stat].values[0]
                inflation_adjustment_stat = inflation_adjustment_league_avg[stat].values[0]
                
                if league_avg_stat != 0:
                    adjustment = inflation_adjustment_stat / league_avg_stat
                else:
                    adjustment = None
                
                inflation_adjustments[stat].append({
                    'Season': season,
                    stat: adjustment
                })

# Convert the result to a DataFrame for each stat
inflation_adjustment_dfs = {}
for stat in stats:
    inflation_adjustment_dfs[stat] = pd.DataFrame(inflation_adjustments[stat])

#Condense into one data frame to display as table
final_df = inflation_adjustment_dfs['PTS']
for stat in stats:
    if stat != 'PTS':
        final_df = pd.merge(final_df, inflation_adjustment_dfs[stat], on='Season')
    

# Round all the stats to 2 decimal places
for stat in stats:
    final_df[stat] = final_df[stat].round(2)

# Display the results for each stat
print(f"Inflation Adjustment Factors (Adjusted to {inflation_adjustment_season} Averages):")
print(tabulate(final_df, headers='keys', tablefmt='grid', showindex=False))


Inflation Adjustment Factors (Adjusted to 2023-24 Averages):
+----------+-------+-------+-------+-------+-------+-------+
| Season   |   PTS |   TRB |   AST |   STL |   BLK |   TOV |
+==========+=======+=======+=======+=======+=======+=======+
| 2008-09  |  1.06 |  0.98 |  1.19 |  0.95 |  1    |  0.9  |
+----------+-------+-------+-------+-------+-------+-------+
| 2009-10  |  1.07 |  0.98 |  1.18 |  0.97 |  1    |  0.9  |
+----------+-------+-------+-------+-------+-------+-------+
| 2010-11  |  1.07 |  0.98 |  1.16 |  0.95 |  1    |  0.89 |
+----------+-------+-------+-------+-------+-------+-------+
| 2011-12  |  1.1  |  0.96 |  1.18 |  0.9  |  0.95 |  0.87 |
+----------+-------+-------+-------+-------+-------+-------+
| 2012-13  |  1.09 |  0.97 |  1.13 |  0.89 |  0.95 |  0.87 |
+----------+-------+-------+-------+-------+-------+-------+
| 2013-14  |  1.08 |  0.97 |  1.16 |  0.93 |  1.04 |  0.88 |
+----------+-------+-------+-------+-------+-------+-------+


### Inflation Adjusted Per 75 Possessions Stats (Year-by-Year)

Running this cell will calculate and display the inflation adjusted per 75 possessions stats for each season in the specified range. The formula for year-by-year inflation adjusted per 75 possessions stats is shown below.

IA stat/75 = (total stat * inflation adjustment factor * 75 * 48) / (minutes played * team possessions per game)

Additionally, this cell calculates and displays relative true shooting percentage, or rTS%, the metric most commonly used for comparing player scoring efficiency across eras. The formula for relative true shooting percentage is below.

rTS% = TS% - league average TS% = ((total points * 100) / (2 * (total field goals attempted + 0.44 * total free throws attemped))) - ((league total points * 100) / (2 * (league total field goals attempted + 0.44 * league total free throws attemped)))

It will display these stats as a table so you can easily see how your selected player performed when pace, minutes, and league scoring are held constant. No action is required on your part other than running the cell.

In [1404]:
from IPython.display import display, HTML

# List of stats to calculate inflation adjustment for
stats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

# Dictionary to hold the final adjusted stats
adjusted_stats = {stat: [] for stat in stats}
adjusted_true_shooting = []

# Loop through each season in the specified range
for season in filtered_player_stats['Season'].unique():
    league_avg_season = league_averages_per100[league_averages_per100['Season'] == season]
    possessions_row = team_possessions_per_game_df[team_possessions_per_game_df['Season'] == season]

    if not league_avg_season.empty and not possessions_row.empty:
        possessions_per_game = possessions_row['Team Possessions Per Game'].values[0]

        for stat in stats:
            adjustment_df = inflation_adjustment_dfs[stat]
            adjustment_row = adjustment_df[adjustment_df['Season'] == season]

            if not adjustment_row.empty:
                adjustment = adjustment_row[stat].values[0]

                if adjustment is not None:
                    for _, player in filtered_player_stats[filtered_player_stats['Season'] == season].iterrows():
                        minutes_played = player['MP']
                        total_stat = player[stat]

                        if minutes_played > 0 and possessions_per_game > 0:
                            adjusted_stat = (total_stat * adjustment * 75 * 48) / (minutes_played * possessions_per_game)
                            adjusted_stats[stat].append({
                                'Season': season,
                                f"IA {stat}/75": adjusted_stat
                            })

        # Calculate relative true shooting percentage
        league_avg_ts = league_avg_season['TS%'].values[0] if 'TS%' in league_avg_season.columns else None
        if league_avg_ts is not None:
            for _, player in filtered_player_stats[filtered_player_stats['Season'] == season].iterrows():
                total_points = player['PTS']
                total_fga = player['FGA']
                total_fta = player['FTA']

                if (total_fga + 0.44 * total_fta) > 0:
                    player_ts = (total_points * 100) / (2 * (total_fga + 0.44 * total_fta))
                    relative_ts = player_ts - league_avg_ts * 100
                    relative_ts_str = f"{relative_ts:+.2f}%"  # Format with + or - sign
                    adjusted_true_shooting.append({
                        'Season': season,
                        'rTS%': relative_ts_str
                    })

# Convert the result to DataFrames
adjusted_stats_dfs = {}
for stat in stats:
    adjusted_stats_dfs[stat] = pd.DataFrame(adjusted_stats[stat])

adjusted_true_shooting_df = pd.DataFrame(adjusted_true_shooting)

#Condense into one data frame to display as table
final_per75 = filtered_player_stats[['Season','Tm','G','MP',]]
for stat in stats:
    final_per75 = pd.merge(final_per75, adjusted_stats_dfs[stat], on='Season')
        
final_per75 = pd.merge(final_per75, adjusted_true_shooting_df, on='Season')

# Round all the stats to 2 decimal places
for stat in stats:
    final_per75[f"IA {stat}/75"] = final_per75[f"IA {stat}/75"].round(2)
    
#Calculate minutes per game instead of minutes
final_per75 = final_per75.rename(columns = {'MP':'MPG'})
final_per75['MPG'] = final_per75['MPG']/final_per75['G']
final_per75['MPG'] = final_per75['MPG'].round(1)

final_per75['G'] = final_per75['G'].astype(int)
    
# Display the results for each stat
final_per75 = final_per75.rename(columns = {'IA TOV/75':'IA TO/75','IA TRB/75':'IA RB/75'})
print(f"Inflation Adjusted Stats Per 75 Possessions (Adjusted to {inflation_adjustment_season} Averages) and Relative True Shooting Percentage by Season:")
print(tabulate(final_per75, headers='keys', tablefmt='pretty', showindex=False))
final_per75 = final_per75.rename(columns = {'IA TO/75':'IA TOV/75','IA RB/75':'IA TRB/75'})

Inflation Adjusted Stats Per 75 Possessions (Adjusted to 2023-24 Averages) and Relative True Shooting Percentage by Season:
+---------+-----+----+------+-----------+----------+-----------+-----------+-----------+----------+---------+
| Season  | Tm  | G  | MPG  | IA PTS/75 | IA RB/75 | IA AST/75 | IA STL/75 | IA BLK/75 | IA TO/75 |  rTS%   |
+---------+-----+----+------+-----------+----------+-----------+-----------+-----------+----------+---------+
| 2008-09 | CLE | 81 | 37.7 |   32.58   |   8.0    |   9.24    |   1.73    |   1.24    |   2.88   | +4.73%  |
| 2009-10 | CLE | 76 | 39.0 |   32.15   |   7.23   |   10.2    |   1.62    |   1.02    |   3.13   | +6.14%  |
| 2010-11 | MIA | 79 | 38.8 |   29.34   |   7.51   |   8.31    |   1.52    |   0.65    |   3.27   | +5.31%  |
| 2011-12 | MIA | 62 | 37.5 |   31.5    |   8.01   |   7.75    |   1.76    |    0.8    |   3.14   | +7.84%  |
| 2012-13 | MIA | 76 | 37.9 |   30.65   |   8.15   |   8.57    |   1.59    |   0.88    |   2.72   | +10.55

### Inflation Adjusted Per 75 Possessions Stats (Total and Cumulative)

Running this cell will calculate and display the inflation adjusted per 75 possessions stats over all seasons in the specified range. The formula for total inflation adjusted per 75 possessions stats over multiple years is shown below.

IA stat/75 = ($\sum_{years}$(total stat * inflation adjustment factor * 75 * 48)) / (($\sum_{years}$minutes played) * team possessions per game weighted average by minutes)

Additionally, this cell calculates and displays the cumulative inflation adjusted stats, for which the formula is below. These are similar to standard cumulative stats (e.g. the scoring record LeBron James broke in 2023 that was 38,387 total points), just with an inflation adjustment for each season.

IA stat = $\sum_{years}$(total stat * inflation adjustment factor)

Additionally, this cell calculates and displays the total relative true shooting percentage over all seasons in the sepcified range. The formula for relative true shooting percentage across multiple years is below.

rTS% = TS% - league average TS% = (($\sum_{years}$total points * 100) / (2 * ($\sum_{years}$total field goals attempted + 0.44 * $\sum_{years}$total free throws attemped))) - (($\sum_{years}$league total points * 100) / (2 * ($\sum_{years}$league total field goals attempted + 0.44 * $\sum_{years}$league total free throws attemped)))

It will display these stats below so you can easily see how your selected player performed over a multi-year span when pace, minutes, and league scoring are held constant. No action is required on your part other than running the cell.

In [1405]:
import math

# List of stats to calculate inflation adjustment for
stats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

# Dictionary to hold the cumulative stats
cumulative_stats = {stat: 0 for stat in stats}

# Cumulative variables for later calculations
cumulative_minutes_played = 0
cumulative_possessions_weighted = 0
total_games_played = 0
cumulative_points = 0
cumulative_fga = 0
cumulative_fta = 0
cumulative_ts_attempts_weighted = 0
cumulative_ts_weighted_avg = 0

# Cumulative variables specifically for TOV starting from 1977-78
cumulative_minutes_played_tov = 0
cumulative_possessions_weighted_tov = 0
total_games_played_tov = 0

# Loop through each season in the specified range
for season in filtered_player_stats['Season'].unique():
    league_avg_season = league_averages_per100[league_averages_per100['Season'] == season]
    possessions_row = team_possessions_per_game_df[team_possessions_per_game_df['Season'] == season]
    if not league_avg_season.empty and not possessions_row.empty:
        possessions_per_game = possessions_row['Team Possessions Per Game'].values[0]
        league_avg_ts = league_avg_season['TS%'].values[0] if 'TS%' in league_avg_season.columns else None

        for stat in stats:
            # Skip calculation for TOV if the season is before 1977-78
            if stat == 'TOV' and season < '1977-78':
                continue

            adjustment_df = inflation_adjustment_dfs[stat]
            adjustment_row = adjustment_df[adjustment_df['Season'] == season]
            adjustment = adjustment_row[stat].values[0] if not adjustment_row.empty else 0

            for _, player in filtered_player_stats[filtered_player_stats['Season'] == season].iterrows():
                games_played = player['G']
                minutes_played = player['MP']
                total_stat = player[stat]

                cumulative_stats[stat] += total_stat * adjustment
                if stat == 'PTS': # Ensures games, minutes and possessions are only added once per season
                    if not(math.isnan(games_played)):
                        total_games_played += games_played
                    cumulative_minutes_played += minutes_played
                    cumulative_possessions_weighted += minutes_played * possessions_per_game
                
                # Ensure games, minutes and possessions for TOV are only added for 1977-78 or later
                if stat == 'TOV' and season >= '1977-78':
                    cumulative_minutes_played_tov += minutes_played
                    cumulative_possessions_weighted_tov += minutes_played * possessions_per_game
                    total_games_played_tov += games_played

                # For true shooting percentage
                total_points = player['PTS']
                total_fga = player['FGA']
                total_fta = player['FTA']
                ts_attempts = 2 * (total_fga + 0.44 * total_fta)
                cumulative_points += total_points
                cumulative_fga += total_fga
                cumulative_fta += total_fta
                if league_avg_ts is not None:
                    cumulative_ts_attempts_weighted += ts_attempts
                    cumulative_ts_weighted_avg += league_avg_ts * ts_attempts

# Calculate the final adjustment factor
weighted_average_team_poss_per_game = cumulative_possessions_weighted / cumulative_minutes_played
adjustment_factor = (75 * 48) / (cumulative_minutes_played * weighted_average_team_poss_per_game)

# Calculate the final adjustment factor specifically for TOV
if cumulative_minutes_played_tov > 0:
    weighted_average_team_poss_per_game_tov = cumulative_possessions_weighted_tov / cumulative_minutes_played_tov
    adjustment_factor_tov = (75 * 48) / (cumulative_minutes_played_tov * weighted_average_team_poss_per_game_tov)
else:
    adjustment_factor_tov = 0

# Calculate the final inflation adjusted stats
final_adjusted_stats = {}
for stat in stats:
    if stat == 'TOV':
        final_adjusted_stats[stat] = (cumulative_stats[stat] * adjustment_factor_tov)
    else:
        final_adjusted_stats[stat] = (cumulative_stats[stat] * adjustment_factor)

# Calculate the final relative true shooting percentage
player_ts = (cumulative_points * 100) / (2 * (cumulative_fga + 0.44 * cumulative_fta))
league_avg_ts_weighted = cumulative_ts_weighted_avg / cumulative_ts_attempts_weighted
relative_ts = player_ts - league_avg_ts_weighted * 100
relative_ts_str = f"{relative_ts:+.2f}%"  # Format with + or - sign

# Display the final results
print(f"Games Played: {total_games_played}")
print(f"Minutes Played: {cumulative_minutes_played}")
print(f"Minutes Per Game: {round(cumulative_minutes_played/total_games_played,2)}")

print(f"\nInflation Adjusted Stats Per 75 Possessions from {start_season} to {end_season} (Adjusted to {inflation_adjustment_season} Averages)")
for stat in stats:
    print(f"{stat}: {final_adjusted_stats[stat]:.2f}")
    
print(f"\nCumulative Inflation Adjusted Stats from {start_season} to {end_season} (Adjusted to {inflation_adjustment_season} Averages)")
for stat in stats:
    print(f"{stat}: {cumulative_stats[stat]:.2f}")

print("\nRelative True Shooting Percentage:")
print(relative_ts_str)


Games Played: 451.0
Minutes Played: 17188.0
Minutes Per Game: 38.11

Inflation Adjusted Stats Per 75 Possessions from 2008-09 to 2013-14 (Adjusted to 2023-24 Averages)
PTS: 31.14
TRB: 7.64
AST: 8.66
STL: 1.62
BLK: 0.83
TOV: 3.06

Cumulative Inflation Adjusted Stats from 2008-09 to 2013-14 (Adjusted to 2023-24 Averages)
PTS: 13474.36
TRB: 3307.11
AST: 3748.61
STL: 700.67
BLK: 357.66
TOV: 1325.66

Relative True Shooting Percentage:
+7.38%


## **Inflation and Pace Adjusted Per Game (IPAPG) Stats Intro**

One additional thing I wanted to do is calculate another version of these stats that don't hold minutes constant for different players. You see, inflation adjusted per 75 stats are considered the gold standard for comparing players across eras because they adjust for differences in team pace and era, but they also effectively adjust for the number of minutes per game a player plays. This can be useful in some circumstances, but there are some drawbacks. For example, a player playing 12 minutes per game may be productive in his role, but would not be able to scale up his production, so he wouldn't be nearly as valuable as a star player with the same inflation adjusted per 75 stats who plays 36 minutes per game. To account for this, I wanted to create a version of inflation adjusted per 75 stats that are multiplied by a player's minutes per game and divided by 36 (standard star minutes that roughly align with 75 possessions). This will result in stats that adjust for the team's pace and the league average of the given stat in the given year, but will reward players who play more minutes per game, as per game stats do. These stats will roughly align with per game stats just with pace and inflation adjustments. Thus, I call them Inflation and Pace Adjusted Per Game stats, or IPAPG stats.



### Inflation and Pace Adjusted Per Game (IPAPG) Stats (Year-by-Year)

As mentioned above, the formula for these stats on a year-by-year basis is: IPA {stat}PG = (IA {stat}/75 * total minutes) / (games played * 36)

The cell below will calculate the given player's Inflation and Pace Adjusted Points Per Game (IPA PPG), Inflation and Pace Adjusted Rebounds Per Game (IPA RPG), Inflation and Pace Adjusted Assists Per Game (IPA APG), Inflation and Pace Adjusted Steals Per Game (IPA SPG), Inflation and Pace Adjusted Blocks Per Game (IPA BPG), and Inflation and Pace Adjusted Turnovers Per Game (IPA TOPG) in each of the seasons from the selected start season to the selected end season. Additionally, it will also display the relative true shooting percentage which is the same as it is in the year-by-year IA /75 stats table.

It will display these stats as a table so you can easily see how your selected player performed when pace and league scoring are held constant but minutes are not. No action is required on your part other than running the cell.

In [1406]:
# Initialize results DataFrame
stats = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']
ipapg_columns = [f'IPA {stat[:1]}PG' for stat in stats]
ipapg_columns[1] = 'IPA RPG'
ipapg_columns[5] = 'IPA TOPG'
final_pergame = pd.DataFrame(columns=['Season','Tm','G','MPG'] + ipapg_columns)

# Calculate IPAPG stats
for season in filtered_player_stats['Season'].unique():
    season_stats = player_stats[player_stats['Season'] == season]
    ia_stats_row = final_per75[final_per75['Season'] == season]
    
    if not ia_stats_row.empty:
        ia_stats = ia_stats_row.iloc[0]  # Get IA/75 stats for the season
        season_results = {'Season': season, 'Tm': ''.join([char for char in season_stats['Tm'] if char.isalpha()]), 'G': season_stats['G'], 'MPG': season_stats['MP']/season_stats['G']}
        
        for stat in stats:
            ia_per75_stat = ia_stats[f'IA {stat}/75']
            total_minutes = season_stats['MP'].sum()
            games_played = season_stats['G'].sum()
            
            if games_played > 0:
                ipapg_stat = (ia_per75_stat * total_minutes) / (games_played * 36)
            else:
                ipapg_stat = 0
            
            stat_char = stat[:1]
            if stat == 'TRB':
                stat_char = 'R'
            if stat == 'TOV':
                stat_char = 'TO'
            season_results[f'IPA {stat_char}PG'] = ipapg_stat
        
        final_pergame = pd.concat([final_pergame, pd.DataFrame([season_results])], ignore_index=True)


# Round all the stats to 2 decimal places
for column in ipapg_columns:
    final_pergame[column] = final_pergame[column].round(2)

# Function to extract the last value from the nested Series for MPG and G columns
def extract_last_value(value):
    if isinstance(value, pd.Series) and not value.empty:
        return value.iloc[-1]
    return value

final_pergame['MPG'] = final_pergame['MPG'].apply(extract_last_value).astype(float).round(1)
final_pergame['G'] = final_pergame['G'].apply(extract_last_value).astype(int)

final_pergame = pd.merge(final_pergame, adjusted_true_shooting_df, on='Season')

# Display the final results using tabulate
print(f"Inflation and Pace Adjusted Per Game Stats (Adjusted to {inflation_adjustment_season} Averages) and Relative True Shooting Percentage by Season:")

print(tabulate(final_pergame, headers = 'keys', tablefmt = 'pretty'))

Inflation and Pace Adjusted Per Game Stats (Adjusted to 2023-24 Averages) and Relative True Shooting Percentage by Season:
+---+---------+-----+----+------+---------+---------+---------+---------+---------+----------+---------+
|   | Season  | Tm  | G  | MPG  | IPA PPG | IPA RPG | IPA APG | IPA SPG | IPA BPG | IPA TOPG |  rTS%   |
+---+---------+-----+----+------+---------+---------+---------+---------+---------+----------+---------+
| 0 | 2008-09 | CLE | 81 | 37.7 |  34.12  |  8.38   |  9.68   |  1.81   |   1.3   |   3.02   | +4.73%  |
| 1 | 2009-10 | CLE | 76 | 39.0 |  34.85  |  7.84   |  11.06  |  1.76   |  1.11   |   3.39   | +6.14%  |
| 2 | 2010-11 | MIA | 79 | 38.8 |  31.6   |  8.09   |  8.95   |  1.64   |   0.7   |   3.52   | +5.31%  |
| 3 | 2011-12 | MIA | 62 | 37.5 |  32.83  |  8.35   |  8.08   |  1.83   |  0.83   |   3.27   | +7.84%  |
| 4 | 2012-13 | MIA | 76 | 37.9 |  32.23  |  8.57   |  9.01   |  1.67   |  0.93   |   2.86   | +10.55% |
| 5 | 2013-14 | MIA | 77 | 37.7 |  32

/var/folders/pr/t9901z0n4z7dbkstysncwp0m0000gn/T/ipykernel_1674/2699811653.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_pergame = pd.concat([final_pergame, pd.DataFrame([season_results])], ignore_index=True)


### Inflation and Pace Adjusted Per Game (IPAPG) Stats (Total and Cumulative)

Running this cell will calculate and display the inflation and pace adjusted per game (IPAPG) stats over all seasons in the specified range. The formula for total IPAPG stats over multiple years is shown below.

IPA {stat}PG = ($\sum_{years}$(total stat * inflation adjustment factor * 75 * 48)) / (36 * ($\sum_{years}$games played) * team possessions per game weighted average by minutes)

Additionally, this cell calculates and displays the cumulative inflation adjusted stats and the total relative true shooting percentage over all seasons in the sepcified range, which are the same as they are in the total and cumulative IA /75 stats section.

It will display these stats below so you can easily see how your selected player performed over a multi-year span when pace and league scoring are held constant but minutes are not. No action is required on your part other than running the cell.

In [1407]:
#Calculate adjustment factor for IPAPG stats
adjustment_factor = (75 * 48) / (36 * total_games_played * weighted_average_team_poss_per_game)

# Calculate the final adjustment factor specifically for TOV
if cumulative_minutes_played_tov > 0:
    weighted_average_team_poss_per_game_tov = cumulative_possessions_weighted_tov / cumulative_minutes_played_tov
    adjustment_factor_tov = (75 * 48) / (36 * total_games_played_tov * weighted_average_team_poss_per_game_tov)
else:
    adjustment_factor_tov = 0

# Calculate the final inflation adjusted stats
final_adjusted_stats = {}
for stat in stats:
    if stat == 'TOV':
        final_adjusted_stats[stat] = (cumulative_stats[stat] * adjustment_factor_tov)
    else:
        final_adjusted_stats[stat] = (cumulative_stats[stat] * adjustment_factor)

# Display the final results
print(f"Games Played: {total_games_played}")
print(f"Minutes Played: {cumulative_minutes_played}")
print(f"Minutes Per Game: {round(cumulative_minutes_played/total_games_played,2)}")

print(f"\nInflation and Pace Adjusted Per Game Stats from {start_season} to {end_season} (Adjusted to {inflation_adjustment_season} Averages)")
for stat in stats:
    print(f"{stat}: {final_adjusted_stats[stat]:.2f}")
    
print(f"\nCumulative Inflation Adjusted Stats from {start_season} to {end_season} (Adjusted to {inflation_adjustment_season} Averages)")
for stat in stats:
    print(f"{stat}: {cumulative_stats[stat]:.2f}")

print("\nRelative True Shooting Percentage:")
print(relative_ts_str)

Games Played: 451.0
Minutes Played: 17188.0
Minutes Per Game: 38.11

Inflation and Pace Adjusted Per Game Stats from 2008-09 to 2013-14 (Adjusted to 2023-24 Averages)
PTS: 32.97
TRB: 8.09
AST: 9.17
STL: 1.71
BLK: 0.88
TOV: 3.24

Cumulative Inflation Adjusted Stats from 2008-09 to 2013-14 (Adjusted to 2023-24 Averages)
PTS: 13474.36
TRB: 3307.11
AST: 3748.61
STL: 700.67
BLK: 357.66
TOV: 1325.66

Relative True Shooting Percentage:
+7.38%
